In [ ]:
import pandas as pd
import mysql.connector
import os

# List of CSV files and their corresponding table names
csv_files = [
    ('customers.csv', 'customers'),
    ('orders.csv', 'orders'),
    ('sellers.csv', 'sellers'),
    ('products.csv', 'products'),
    ('order_items.csv', 'order_items'),
    ('geolocation.csv', 'geolocation'),
    ('payments.csv', 'payments')  # Added payments.csv for specific handling
]

# Connect to the MySQL database
conn = mysql.connector.connect(
    host='Localhost',
    user='project',
    password='Basha@123',
    database='ecommerce'
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = 'L:/Python&Sql project files'

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Replace NaN with None to handle SQL NULL
    df = df.where(pd.notnull(df), None)
    
    # Debugging: Check for NaN values
    print(f"Processing {csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")

    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'`{col}` {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'
    cursor.execute(create_table_query)

    # Insert DataFrame data into the MySQL table
    for _, row in df.iterrows():
        # Convert row to tuple and handle NaN/None explicitly
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)

    # Commit the transaction for the current CSV file
    conn.commit()
# Close the connection
conn.close()



Processing customers.csv
NaN values before replacement:
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

Processing orders.csv
NaN values before replacement:
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

Processing sellers.csv
NaN values before replacement:
seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64

Processing products.csv
NaN values before replacement:
product_id                      0
product category              610
product_name_length           610
product_description_length    610
product_photos_qty            610
prod

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
db = mysql.connector.connect(
    host='Localhost',
    user='project',
    password='Basha@123',
    database='ecommerce'
)
cur = db.cursor()

#  List all unique cities where customers are located.  

In [2]:
query = """select Distinct customer_city from customers """
cur.execute(query)
data = cur.fetchall()
data[:5]

[('franca',),
 ('sao bernardo do campo',),
 ('sao paulo',),
 ('mogi das cruzes',),
 ('campinas',)]

 #   Count the number of orders placed in 2017.

In [ ]:
query = """select count(order_id) from orders where year(order_purchase_timestamp) = 2017"""
cur.execute(query)
data = cur. fetchall()
print("Number of orders placed in 2017: ",data[0][0])

# Find the total sales per category.

In [ ]:
query = """select upper(products.product_category),round(sum(payments.payment_value),2)from products join order_items
on products.product_id = order_items.product_id
join payments 
on payments.order_id = order_items.order_id
group by product_category
"""
cur.execute(query)
data = cur.fetchall()
df=pd.DataFrame(data,columns = ['category','sales'])
df.head(10)


#  Calculate the percentage of orders that were paid in installments.

In [ ]:
query = """ select sum(case when payment_installments >= 1 then 1
else 0 end)/count(*)*100 from payments"""
cur.execute(query)
data = cur.fetchall()
"percentage of orders that were paid in installments",data[0][0]

#  Count the number of customers from each state. 

In [ ]:
query = """ select count(customer_id),customer_state from customers group by customer_state"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns= ["Customer_ID's_count","States"])
df = df.sort_values(by = "Customer_ID's_count",ascending = True)
plt.figure(figsize = (9,4))
plt.bar(df["States"],df["Customer_ID's_count"])
plt.xticks(rotation= 90)
plt.xlabel("States")
plt.ylabel("customer count")
plt.title("States wise customer count")
plt.show()    

#  Calculate the number of orders per month in 2018.


In [ ]:
query = """ select monthname(order_purchase_timestamp) months,count(order_id) order_count from orders 
where year(order_purchase_timestamp)= 2018 group by months"""
cur.execute(query)
data= cur.fetchall()
df = pd.DataFrame(data,columns = ["Months","order_count"])
o = ("January","February","March","April","May","June","July","August","September","October")
gp = sns.barplot(x = df["Months"],y = df["order_count"],data=df,order = o,color= "green")
plt.xticks(rotation = 45)
gp.bar_label(gp.containers[0])
plt.show()

# Find the average number of products per order, grouped by customer city.

In [ ]:
query = """ with count_per_order as (select orders.order_id,orders.customer_id,count(order_items.order_id) as oc
from orders join order_items on order_items.order_id = orders.order_id
group by  orders.order_id,orders.customer_id)

select customers.customer_city,round(avg(count_per_order.oc),2)as average_orders 
from customers join count_per_order on 
count_per_order.customer_id= customers.customer_id
group by customers.customer_city order by average_orders 
"""
cur.execute(query)
data= cur.fetchall()
df = pd.DataFrame(data, columns= ["customer city","average products/order"])
df.tail(10)

 # Calculate the percentage of total revenue contributed by each product category.

In [ ]:
query = """ select upper(products.product_category )as category,
round(sum(payments.payment_value)/(select sum(payments.payment_value))*100 ,2)as sales_percentage
from products join order_items
on products.product_id = order_items.product_id
join payments on 
payments.order_id = order_items.order_id
group by category order by sales_percentage desc"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns=["Category","percentage"])
df.head(10)

 # Identify the correlation between product price and the number of times a product has been purchased.

In [ ]:
query = """ SELECT products.product_category,
count(order_items.product_id),
round(avg(order_items.price),2)
from products join order_items on product_id = order_items.product_id
group by products.product_category"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns=["Category","order_count","price"])
arr1 = df["order_count"]
arr2 = df["price"]
np.corrcoef([arr1,arr2])

# Calculate the total revenue generated by each seller, and rank them by revenue.

In [ ]:
query = """select *,rank() over (order by revenue desc)as rn from 
(select order_items.seller_id,sum(payments.payment_value) revenue
from order_items join payments on 
order_items.order_id = payments.order_id
group by order_items.seller_id)as a"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns = ["seller_id","revenue","rank"])
df = df.head()
gp = sns.barplot(x = "seller_id",y = "revenue",data = df)
plt.xticks(rotation=90)
gp.bar_labels(cotainers[0])
plt.show()


# Calculate the moving average of order values for each customer over their order history.

In [ ]:
query = """ select customer_id, order_purchase_timestamp,payment,avg(payment)
over(partition by customer_id order by order_purchase_timestamp rows between 2 preceding and current row)
as mov_avg
from
(select orders.customer_id,orders.order_purchase_timestamp,payments.payment_value as payment
from payments join orders on payments.order_id = orders.order_id) as a """
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns = ["customer id","Date and Time","payments","avg payments"])
df = df.head()

# Calculate the cumulative sales per month for each year.

In [ ]:
query = """ select years ,months,payment,sum(payment) over(order by years,months) cumulative_sales from
(select year(orders.order_purchase_timestamp)as years,
 month(orders.order_purchase_timestamp)as months,
 round(sum(payments.payment_value),2) as payment from orders join payments on orders.order_id  = payments.order_id
 group by years ,months order by years,months) as a """
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns = ["years","months","payments","cum_sum Payments"])
df.head(5)

# Calculate the year-over-year growth rate of total sales.

In [ ]:
query = """ with a as(select year(orders.order_purchase_timestamp) as years,
round(sum(payments.payment_value),2) as payment from orders join payments
on orders.order_id = payments.order_id
group by years order by years)

select years,((payment-lag(payment,1) over(order by years)) / 
lag(payment,1) over(order by years))*100 from a
"""
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns= ["Year","YoY % Growth "])
df

#   Calculate the retention rate of customers, defined as the percentage of customers who make another purchase within 6 months of their first purchase.

In [ ]:
query = """  with a as (select customers.customer_id,
min(orders.order_purchase_timestamp) first_order
from customers join orders
on customers.customer_id = orders.customer_id
group by customers.customer_id),

b as (select a.customer_id,count(distinct orders.order_purchase_timestamp) next_order
from  a join orders 
on orders.customer_id = a.customer_id
and orders.order_purchase_timestamp > first_order
and orders.order_purchase_timestamp < 
date_add(first_order,interval 6 month)
group by a.customer_id)

select 100 * (count( distinct a.customer_id)/ count(distinct b.customer_id))
from a left join b 
on a.customer_id = b.customer_id """
cur.execute(query)
data = cur.fetchall()
data

# Identify the top 3 customers who spent the most money in each year.

In [ ]:
query = """ select years,customer_id,payment,d_rank from
(select year( orders.order_purchase_timestamp) years,
orders.customer_id,
sum(payments.payment_value) payment,
dense_rank() over(partition by year ( orders.order_purchase_timestamp)
order by sum(payments.payment_value)desc) d_rank
from orders join payments
on payments.order_id = orders.order_id
group by year( orders.order_purchase_timestamp),
orders.customer_id) as a
where d_rank <= 3 ; """
cur.execute(query)
data = cur.fetchall()
df = pd.DataFrame(data,columns=["years","id","payment","rank"])
sns.barplot(x= "id",y = "payment",data= df,hue = "years")
plt.xticks(rotation= 90)
plt.show()

